In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import friedmanchisquare
from cd_plot import cd_evaluation
import scikit_posthocs as sp
import matplotlib.pyplot as plt
import seaborn as sns

import math
import warnings
import matplotlib.pyplot as plt
from autorank._util import RankResult, get_sorted_rank_groups, rank_multiple_nonparametric, test_normality


HP_IDX = "HP_Idx"
OPTIMIZER = "Optimizer"
SUBSPACE = "Subspace"
OPSET = "Opset"
TEST_ACC1 = "Test Accuracy"
BENCHMARK = "Benchmark"

WIDE = "wide"
DEEP = "deep"
SINGLE_CELL = "single_cell"
ALL_SKIP = "all_skip"
NO_SKIP = "no_skip"
REGULAR = "regular"

### Code for Plotting CD Diagrams

In [ ]:
def _custom_cd_diagram(result, reverse, ax, width):
    """
    !TAKEN FROM AUTORANK WITH MODIFICATIONS!
    """

    def plot_line(line, color="k", **kwargs):
        ax.plot([pos[0] / width for pos in line], [pos[1] / height for pos in line], color=color, **kwargs)

    def plot_text(x, y, s, *args, **kwargs):
        ax.text(x / width, y / height, s, *args, **kwargs)

    sorted_ranks, names, groups = get_sorted_rank_groups(result, reverse)
    cd = result.cd

    lowv = min(1, int(math.floor(min(sorted_ranks))))
    highv = max(len(sorted_ranks), int(math.ceil(max(sorted_ranks))))
    cline = 0.4
    textspace = 1
    scalewidth = width - 2 * textspace

    def rankpos(rank):
        if not reverse:
            relative_rank = rank - lowv
        else:
            relative_rank = highv - rank
        return textspace + scalewidth / (highv - lowv) * relative_rank

    linesblank = 0.2 + 0.2 + (len(groups) - 1) * 0.1

    # add scale
    distanceh = 0.1
    cline += distanceh

    # calculate height needed height of an image
    minnotsignificant = max(2 * 0.2, linesblank)
    height = cline + ((len(sorted_ranks) + 1) / 2) * 0.2 + minnotsignificant

    if ax is None:
        fig = plt.figure(figsize=(width, height))
        fig.set_facecolor("white")
        ax = fig.add_axes([0, 0, 1, 1])  # reverse y axis
    ax.set_axis_off()

    # Upper left corner is (0,0).
    ax.plot([0, 1], [0, 1], c="w")
    ax.set_xlim(0, 1)
    ax.set_ylim(1, 0)

    plot_line([(textspace, cline), (width - textspace, cline)], linewidth=0.7)

    bigtick = 0.1
    smalltick = 0.05

    tick = None
    for a in list(np.arange(lowv, highv, 0.5)) + [highv]:
        tick = smalltick
        if a == int(a):
            tick = bigtick
        plot_line([(rankpos(a), cline - tick / 2), (rankpos(a), cline)], linewidth=0.7)

    for a in range(lowv, highv + 1):
        plot_text(rankpos(a), cline - tick / 2 - 0.05, str(a), ha="center", va="bottom")

    for i in range(math.ceil(len(sorted_ranks) / 2)):
        chei = cline + minnotsignificant + i * 0.2
        plot_line([(rankpos(sorted_ranks[i]), cline), (rankpos(sorted_ranks[i]), chei), (textspace - 0.1, chei)], linewidth=0.7)
        plot_text(textspace - 0.2, chei, names[i], ha="right", va="center")

    for i in range(math.ceil(len(sorted_ranks) / 2), len(sorted_ranks)):
        chei = cline + minnotsignificant + (len(sorted_ranks) - i - 1) * 0.2
        plot_line([(rankpos(sorted_ranks[i]), cline), (rankpos(sorted_ranks[i]), chei), (textspace + scalewidth + 0.1, chei)], linewidth=0.7)
        plot_text(textspace + scalewidth + 0.2, chei, names[i], ha="left", va="center")

    # upper scale
    if not reverse:
        begin, end = rankpos(lowv), rankpos(lowv + cd)
    else:
        begin, end = rankpos(highv), rankpos(highv - cd)
    distanceh += 0.15
    bigtick /= 2
    plot_line([(begin, distanceh), (end, distanceh)], linewidth=0.7)
    plot_line([(begin, distanceh + bigtick / 2), (begin, distanceh - bigtick / 2)], linewidth=0.7)
    plot_line([(end, distanceh + bigtick / 2), (end, distanceh - bigtick / 2)], linewidth=0.7)
    plot_text((begin + end) / 2, distanceh - 0.05, "CD", ha="center", va="bottom")

    # no-significance lines
    side = 0.05
    no_sig_height = 0.1
    start = cline + 0.2
    for l, r in groups:
        plot_line([(rankpos(sorted_ranks[l]) - side, start), (rankpos(sorted_ranks[r]) + side, start)], linewidth=2.5)
        start += no_sig_height

    return ax


def cd_evaluation(performance_per_dataset, maximize_metric, output_path=None, ignore_non_significance=False, plt_title=None):
    """Performance per dataset is  a dataframe that stores the performance (with respect to a metric) for  set of
    configurations / models / algorithms per dataset. In  detail, the columns are individual configurations.
    rows are datasets and a cell is the performance of the configuration for  dataset.
    """

    # -- Preprocess data for autorank
    if maximize_metric:
        rank_data = performance_per_dataset.copy() * -1
    else:
        rank_data = performance_per_dataset.copy()
    rank_data = rank_data.reset_index(drop=True)
    rank_data = pd.DataFrame(rank_data.values, columns=list(rank_data))

    # -- Settings for autorank
    alpha = 0.05
    effect_size = None
    verbose = True
    order = "ascending"  # always due to the preprocessing
    alpha_normality = alpha / len(rank_data.columns)
    all_normal, pvals_shapiro = test_normality(rank_data, alpha_normality, verbose)

    # -- Friedman-Nemenyi
    res = rank_multiple_nonparametric(rank_data, alpha, verbose, all_normal, order, effect_size, None)

    result = RankResult(
        res.rankdf,
        res.pvalue,
        res.cd,
        res.omnibus,
        res.posthoc,
        all_normal,
        pvals_shapiro,
        None,
        None,
        None,
        alpha,
        alpha_normality,
        len(rank_data),
        None,
        None,
        None,
        None,
        res.effect_size,
        None,
    )
    print(res.rankdf)
    if result.pvalue >= result.alpha:
        if ignore_non_significance:
            warnings.warn("Result is not significant and results of the plot may be misleading!")
        else:
            raise ValueError("Result is not significant and results of the plot may be misleading. If you still want to see the CD plot, set" +
                             " ignore_non_significance to True.")

    # -- Plot
    fig, ax = plt.subplots(figsize=(8, 4))
    plt.rcParams.update({"font.size": 16})
    _custom_cd_diagram(result, order == "ascending", ax, 8)
    if plt_title:
        plt.title(plt_title)
    plt.tight_layout()
    if output_path:
        plt.savefig(output_path, transparent=True, bbox_inches="tight")
    plt.show()
    plt.close()

    return result


### Load the dataset and make it pretty

In [ ]:
# Load the dataset and make it pretty
df = pd.read_csv("model_trains_full.csv")  # Update with your filename
df[TEST_ACC1] = df["TestAcc1"]
df[OPTIMIZER] = df[OPTIMIZER].replace(
    {
        "drnas": "DrNAS",
        "darts": "DARTS",
        "pcdarts": "PC-DARTS",
        "sdarts": "SmoothDARTS",
        "gdas": "GDAS",
        "oles": "OLES",
        "fairdarts": "FairDARTS",
    }
)
df[SUBSPACE] = df[SUBSPACE].replace(
    {
        "wide": "Wide",
        "deep": "Deep",
        "single_cell": "Single Cell",
    }
)

df[OPSET] = df[OPSET].replace(
    {
        "all_skip": "All Skip",
        "no_skip": "No Skip",
        "regular": "Regular",
    }
)

df[BENCHMARK] = df[SUBSPACE] + " + " + df[OPSET]


### Plot Box Plot

In [ ]:
def box_plot(df, by="Benchmark"):
    # Create a benchmark column for easier handling
   
    # Aggregate statistics
    summary = df.groupby([by, "Optimizer"])["Test Accuracy"].agg(
        mean_accuracy="mean",
        std_accuracy="std",
        median_accuracy="median",
        min_accuracy="min",
        max_accuracy="max"
    ).reset_index()

    print(summary.head())

    # Plot boxplots for each benchmark
    benchmarks = sorted(df[by].unique())

    # Create a grid of subplots for all benchmarks
    num_benchmarks = len(benchmarks)
    cols = 3  # Number of columns in the grid
    rows = (num_benchmarks + cols - 1) // cols  # Calculate the number of rows needed

    fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows), squeeze=True)
    axes = axes.flatten()  # Flatten the axes array for easy iteration

    for i, benchmark in enumerate(benchmarks):
        ax = axes[i]
        optimizers_order = sorted(df["Optimizer"].unique())
        sns.boxplot(
            data=df[df[by] == benchmark],
            x="Optimizer",
            y="Test Accuracy",
            order=optimizers_order,
            ax=ax,
            palette="Set2"  # Assign different colors to each optimizer
        )
        ax.set_title(f"{benchmark}")
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
        ax.set_xlabel("Optimizer")
        ax.set_ylabel("Test Accuracy")

    # Hide any unused subplots
    for j in range(i + 1, len(axes)):
        axes[j].axis("off")

    plt.tight_layout()
    plt.savefig(f"plots/boxplot_{by}.png")
    plt.show()


# Box plot for all benchmarks
box_plot(df, by=BENCHMARK)


#### An example of the friedmanchisquare API

In [ ]:
def friedman_example():
    rng = np.random.default_rng()
    x = rng.random((9, 7))
    # x = np.arange(100).reshape(-1, 10).tolist()
    # print(x.shape)
    res = friedmanchisquare(*x)
    return res.statistic, res.pvalue

for i in range(10):
    print(friedman_example())

#### Compute the Friedman statistics for every benchmark and display it as a LateX table

In [ ]:
def get_benchmark_rows(df, benchmark):
    return df[df[BENCHMARK] == benchmark].reset_index(drop=True)

results = []

for benchmark in df[BENCHMARK].unique():
    rows = get_benchmark_rows(df, benchmark).pivot(index=HP_IDX, columns=OPTIMIZER, values=TEST_ACC1)
    values = rows.to_numpy().tolist()
    res = friedmanchisquare(*values)
    results.append((benchmark, res.statistic, res.pvalue))

friedman_results = pd.DataFrame(results, columns=["Benchmark", "Friedman Stat", "p-value"])
print(friedman_results.to_latex(index=False, float_format="%.8f"))

#### Get the maximum test accuracy for every optimizer x benchmark combination and plot the CD diagram
This should yield 9 benchmarks x 7 optimizers

In [ ]:
# Get the MAX test accuracy for each optimizer and benchmark and pivot the dataframe
# to have optimizers as columns and benchmarks as rows
grouped_df = df.groupby([OPTIMIZER, BENCHMARK])[TEST_ACC1].max().reset_index()
pivot_df = grouped_df.pivot(index=BENCHMARK, columns=OPTIMIZER, values=TEST_ACC1)

cd_evaluation(pivot_df, maximize_metric=True, output_path="plots/cd_plot.jpg", ignore_non_significance=False)
pivot_df

#### Get the best HP Id per Optimizer x Benchmark combination

In [ ]:
df.iloc[df.groupby([OPTIMIZER, BENCHMARK])[TEST_ACC1].idxmax()].groupby([BENCHMARK, OPTIMIZER])[HP_IDX].mean().reset_index().to_csv("best_hp_for_opt_and_benchmark.csv", index=False)

#### Plot the Correlation Heatmap

In [ ]:
def plot_correlation_heatmap(df, method='spearman', figsize=(12, 10)):
    """
    Plot correlation heatmap between optimizers.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame with benchmarks as rows and optimizers as columns
    method : str, default='spearman'
        Correlation method ('spearman' or 'pearson')
    figsize : tuple, default=(12, 10)
        Figure size
    """
    # Calculate correlation between optimizers (columns)
    corr = df.corr(method=method)
    
    # Create a mask for the upper triangle
    mask = np.triu(np.ones_like(corr, dtype=bool))
    
    # Set up the matplotlib figure
    plt.figure(figsize=figsize)
    
    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(
        corr, 
        # mask=mask,
        cmap="viridis",
        vmax=1.0,
        vmin=-1.0,
        center=0,
        square=True,
        linewidths=.5,
        cbar_kws={"shrink": .5,}, # "label": f"{method.capitalize()} Correlation"},
        annot=True,  # Show correlation values
        fmt=".2f",    # Round to 2 decimal places
        # fontsize=50,
    )
    
    plt.title(f"{method.capitalize()} Rank Correlation Between Benchmarks", fontsize=25)
    plt.xticks(rotation=45, ha='right')
    # plt.tight_layout()
    plt.savefig("plots/correlation_heatmap.png", dpi=300, bbox_inches="tight")
    plt.show()
    
    return corr

# Example usage:
# corr_spearman = plot_correlation_heatmap(pivot_df.transpose(), method='spearman')
corr_pearson = plot_correlation_heatmap(pivot_df.transpose(), method='kendall')
pivot_df.transpose()


#### Generate the Win Rate matrix

In [ ]:
# Initialize empty Condorcet matrix
optimizers = pivot_df.columns
condorcet = pd.DataFrame(0, index=optimizers, columns=optimizers, dtype=int)

# Loop through all pairs of optimizers
for i in optimizers:
    for j in optimizers:
        if i != j:
            condorcet.loc[i, j] = (pivot_df[i] > pivot_df[j]).mean()

print("Condorcet matrix (raw wins):")
condorcet

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(condorcet, annot=True, fmt=".2f", cmap="viridis", cbar_kws={"shrink": 0.5})
plt.title("Win Rate", fontsize=20)
# plt.xlabel("Optimizer")
# plt.ylabel("Benchmark")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig("plots/condorcet_matrix.png", dpi=300, bbox_inches="tight")
plt.show()